In [1]:
class Node:
    """This class represents one node of a trie tree.
    
    Attributes
    ----------
    value: str
        Letter represented by the node.
    isendnode: bool
        True if a node marks the end of a word; false otherwise.
    children: dict
        Keys are letters and values are nodes
    frequency: int
        A counter of how many times an end node is inserted into trie
    """

    def __init__(self,value,isendnode):
        '''Creates the Node instance.'''
        
        self.value = value                       
        self.isendnode = isendnode
        #store each node's children in a dictionary 
        #dictionary keys are letters and values are nodes
        self.children = {} 
        #a counter of how many times a word is inserted into the trie
        #this only applies when the node is an end node
        #non-end node's will not have a frequency counter
        self.frequency = 0
                           
        
class Trie:
    """This class represents the entirety of a trie tree.
    
    Attributes
    ----------
    root: Node
        An empty root node.
    word_freq: dict
        Keys are words in the trie, and values are their frequency
    
    Methods
    -------
    insert(self, word)
        Inserts a word into the trie, creating nodes as required.
    lookup(self, word)
        Determines whether a given word is present in the trie.
    helper_recursive(self, node, word)
        Recursively finds words and stores their frequencies in a dictionary (word_freq).
    autocomplete(self, prefix)
        Finds the most common word with the given prefix.
    """
    
    def __init__(self, word_list = None):
        """Creates the Trie instance, inserts initial words if provided.
        
        Parameters
        ----------
        word_list : list
            List of strings to be inserted into the trie upon creation.
        """
        #create an empty node as trie's root node
        self.root = Node('',False)
        
        #empty dictionary where keys are words (str) and values are their frequencey (self.counter)
        self.word_freq = {}
        
        #if there are items in word_list
        if word_list is not None: 
            #insert each word in word_list using insert() method
            for word in word_list:                            
                self.insert(word)
                
                
    def insert(self, word):
        """Inserts a word into the trie, creating missing nodes on the go.
        
        Parameters
        ----------
        word : str
            The word to be inserted into the trie.
        """
        #initialize node to be the root node of trie
        node = self.root                       
        
        #for-loop iterates through letters in a word 
        for l in word:                          
            l = l.lower()  #ignore uppercase letters                    
            
            if l in node.children:   
                #if the letter is already a child of the current node
                #move to that node by setting node to the relevant child node
                node = node.children[l]
                
            else: 
                #if current letter is not found, create a new node with letter as it's value
                new_node = Node(l,False)
                #make new node a child of the current node
                node.children[l] = new_node
                #then move to this new node for the next iteration
                node = new_node                  
        
        #after inserting all letters, mark the last letter (node) as the end of the word
        node.isendnode = True  
        
        #after a word is inserted, increment it's frequency by 1 to record the occurence of word
        node.frequency += 1

        
        
    def lookup(self, word):
        """Determines whether a given word is present in the trie.
        
        Parameters
        ----------
        word : str
            The word to be looked-up in the trie.
            
        Returns
        -------
        bool
            True if the word is present in trie; False otherwise.
            
        Notes
        -----
        Your trie should ignore whether a word is capitalized.
        E.g. trie.insert('Prague') should lead to trie.lookup('prague') = True
        """
        #set node to root of tree
        node = self.root                    
        
        #iterate through every letter
        for l in word: 
            
            if l in node.children:
                #if letter is present in node's children, move to that node
                node = node.children[l]
            else: 
                #if letter isn't in node's children, trie does not contain
                #word, so return False
                return False                
        
        #after finding the word, check if last letter of word is an end node
        #to make sure the word isn't just a prefix
        if node.isendnode == False:  
            #if last node is not marked as end node, return false because word is just a prefix
            return False                     
        
        return True
    
    
    def helper_recursive(self, node, word=''):
        '''Recursively finds words and stores their frequencies in a dictionary (word_freq).
        
        '''
        #since each node stores only one letter and its path from root represents words
        #concatenate letters found during recursion to incrementally form the respective words
        word += node.value                              
        
        #if an end node is found during recursion, concatenated letters are a word
        #store this word in self.word_freq
        if node.isendnode == True:                       
            self.word_freq[word] = node.frequency


        if node.children: 
            #if node has children, recursively call the function to find words and store them
            #in self.word_Freq with their frequency counter
        
            for (i, j) in sorted(node.children.items()):
                self.helper_recursive(j, word)                  

    
    def autocomplete(self, prefix):
        """Finds the most common word with the given prefix.

        You might want to reuse some functionality or ideas from Q4.

        Parameters
        ----------
        prefix : str
            The word part to be “autocompleted”.

        Returns
        ----------
        str
            The complete, most common word with the given prefix.
            
            The return value is equal to prefix if there is no valid word in the trie.
            The return value is also equal to prefix if prefix is the most common word.
        """
        
        #empty out word_freq to avoid duplicates while re-running autocomplete()
        #or while running k_most_common() and autocomplete(), which both use word_freq
        self.word_freq = {} 
        
        #create empty list to store words that start with the given prefix
        prefix_words = []
        
        #obtain words and their frequencies by calling recursive helper function to fill word_freq
        #with all words in trie and their frequencies of occurrence
        if self.root.children:                                                                    
            for (i, j) in sorted(self.root.children.items()): 
                self.helper_recursive(j) 
                
        #sort word_freq by word frequencies in descending order
        sorted_word_freq = sorted(self.word_freq.items(), key=lambda x: x[1], reverse=True)

        #iterate through sorted_word_freq to find words that start with a given prefix
        for (i,j) in sorted_word_freq: #i is word (str) and j is word frequency (number)
            
            #if words starting with the prefix are found, store the word and it's frequency in pref_fil
            if i[0:len(prefix)] == prefix:
                prefix_words.append((i,j))
        
        #if after checking all items in word_freq, valid words aren't found, return prefix
        if prefix_words == []:
            return prefix
        
        #use operator module to find and return the word with the highest frequency in prefix_words 
        from operator import itemgetter
        return max(prefix_words,key=itemgetter(1))[0] 
        


In [2]:
import urllib.request
response = urllib.request.urlopen('http://bit.ly/CS110-Shakespeare')
bad_chars = [';', ',', '.', '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_', '[', ']']

trieSH = Trie()

for line in response:
    line = line.decode(encoding = 'utf-8')
    line = filter(lambda i: i not in bad_chars, line)
    words = "".join(line).split()
    for word in words:
        trieSH.insert(word)

assert trieSH.autocomplete('hist') == 'history'
assert trieSH.autocomplete('en') == 'enter'
assert trieSH.autocomplete('cae') == 'caesar'
assert trieSH.autocomplete('gen') == 'gentleman'
assert trieSH.autocomplete('pen') == 'pen'
assert trieSH.autocomplete('tho') == 'thou'
assert trieSH.autocomplete('pent') == 'pentapolis'
assert trieSH.autocomplete('petr') == 'petruchio'